In [1]:
import sqlite3
import numpy as np
import pandas as pd
from scipy.stats import fisk

In [33]:
class PitStop():
    def __init__(self, team, gp_location, season, dataframes):
        self.team = team.name
        self.gp_location = gp_location
        self.len_train_df = 2
        
        self.season = season
        self.dfs = dataframes
        
        # Retrieve the race_id for the given season and location
        df_races = self.dfs["races"]
        race_row = df_races[
            (df_races["season"] == self.season) & (df_races["location"] == self.gp_location)
        ]

        
        if race_row.empty:
            raise ValueError(f"No race found for location '{self.gp_location}' in season {self.season}.")
        self.race_id = race_row["id"].iloc[0]

    def calculate_best_pit_stop_duration(self):
        df_laps=self.dfs["laps"]
        df_races = self.dfs['races']
        location = df_races[df_races['id']==self.race_id]['location'].iloc[0]
        seasons_to_train = [self.season - x for x in range (1, self.len_train_df +1)]
        races_to_train = list(df_races[(df_races['location']==location) & (df_races['season'].isin(seasons_to_train))]['id'])

        min_pit_stop_duration_per_race = df_laps[df_laps['race_id'].isin(races_to_train)].dropna(subset=['pitstopduration']).groupby(['race_id'])[['pitstopduration']].quantile(q=0.025)
        avg_min_pit_stop_duration = min_pit_stop_duration_per_race['pitstopduration'].mean()
        self.avg_min_pit_stop_duration = avg_min_pit_stop_duration

    def calibrate_pit_stop_variability_law(self):
        df_laps=self.dfs["laps"]
        df_staterfields = self.dfs["starterfields"]
        df_races = self.dfs["races"]

        df_laps_with_season = df_laps.merge(
            df_races[["id", "season"]], 
            left_on="race_id",
            right_on="id",
            how="left"
        ).drop(columns=["id"])  # We can drop the duplicated "id" column
        
        df_merged = df_laps_with_season.merge(
            df_staterfields[["race_id", "driver_id", "team"]],
            on=["race_id", "driver_id"],
            how="left"
        )
        
        df = df_merged[
            (df_merged['team'] == self.team) & 
            (df_merged['season'] == self.season) &
            (df_merged['race_id'] < self.race_id) &
            (df_merged['pitstopduration'].notna()) & 
            (df_merged['pitstopduration'] < 700)
        ].copy()
        
        df[["pitstop_diff"]]=df[['pitstopduration']]- self.avg_min_pit_stop_duration
        shape, loc, scale=fisk.fit(df["pitstop_diff"])
        return [shape, loc, scale]

    def calculate_pit_stop_duration(self):
        shape, loc, scale = self.calibrate_pit_stop_variability_law()
        variability = fisk.rvs(shape, loc=loc, scale=scale, size=1, random_state=None)[0]
        pit_stop = self.avg_min_pit_stop_duration + variability
        return pit_stop
        


In [200]:


    # def run(self):

        
    #     # # Creer un objet Race (attribut = Recap_Tour / Positions -> liste / Drivers_Alive  -> dico / Safety_Car_Laps -> liste ... )
    #     # # Methodes (compute_lap_time(Driver, lap_number) -> cette fonction check si y'a une SC et si le driver est alive et lance le modele / Change les attributs de positions)
    #     self.get_parameters_for_simulation()
    #     # for i in range(1,no_laps+1) : 
    #     #     for driver in list_all_driver :  (list_all_driver est une liste d'objets driver)
    #     #         driver.change_status(i)
    #     #         # Normal Lap
    #     #         time = self.compute_lap_time(driver, i)
    #     #         driver.cumul_time += time
    #     #         # Pit stop :
    #     #         driver.cumul_time += self.pit_stop(driver)
    #                 # driver.update_info()

    #         # # Overtaking
    #         # # Aggrémenter le recap des tours
    #     print("Winner  :  ")
    #     return(self.laps_recap)  

  


    # def pit_stop(self,driver):
    #     #     Dico ={1: { "compound" : , "pitstop_interval" :, "pit_stop_lap" : aléatoire dans pitstop_interval}} # Le dico est un input et sera mis dans driver.pit_stops_informations
    #     # - Check si i == pitstop_lap : # Récupérer pit_stop_lap dans le dico
            
    #     #     driver.age_tire = 0
    #     #     driver.compound = # dans le dico de strategy avec la clé [driver.next_pitstop] et la deuxième clé [compound] 
    #     #     driver.cumul_time += driver.team.calculate_pit_stop_time() # Driver a un attribut qui est un objet Team de la classe Team 
    #     #     driver.next_pitstop += 1
    #     # - check si i isin race.Safety_car_laps and i isin pitstop_interval : 
    #     #     pitstop_lap = i # Changer la valeur dans le dico
    #     #     driver.age_tire = 0
    #     #     driver.compound = # dans le dico de strategy avec la clé [driver.next_pitstop] et la deuxième clé [compound] 
    #     #     driver.cumul_time += driver.team.calculate_pit_stop_time() # Driver a un attribut qui est un objet Team de la classe Team 
    #     #     driver.next_pitstop += 1 
    

    def pit_stop(self,driver):
        """

        Returns:
            float: Duration of the pit stop in seconds.
        """
        
        pit_stop = PitStop(
        team=driver.team.name, 
        gp_location=self.gp_location,
        season=self.gp_location
        dataframes=self.dataframes
    )

        pit_stop.calculate_best_pit_stop_duration() # Peut etre mettre dans team et le récupérer 
        calculated_duration = pit_stop.calculate_pit_stop_duration()
        # Reset tire attributes
        driver.tire_age = 0
        driver.compound = driver.pit_stops_info[driver.next_pit_stop]["compound"]
        driver.next_pit_stop += 1

        return pit_duration


    # def compute_lap_time(self,driver,nolap):
    #     lap_time=0
    #     if driver.alive:
    #         # FUel & TIre Model
    #         model=driver.model
    #         data_for_predictions=driver.select_features(nolap)
    #         lap_time+=model.predict(data_for_predictions) # Fuel level, tire age, coupound

    #     if nolap.isin(self.safetycar_laps):
    #         lap_time*=1.2

    #     return(lap_time)

if __name__ == "__main__":

    data_loader = DataLoader(db_path="F1_timingdata_2014_2019.sqlite")
    dataframes = data_loader.load_data()

    # Sélectionner une saison et un nom de GP
    season_test = 2019
    gp_location_test = "SaoPaulo"

    run_simulation = Run(
        season=season_test,
        gp_location=gp_location_test,
        dataframes=dataframes
    )


    # Vérifier les sorties
    print("\n=== Test de la classe Run ===")
    print("Nom de la course  :", gp_location_test)
    print("Saison            :", season_test)
    print("Nombre de tours   :", run_simulation.number_of_laps)
    print("Grille de départ  :", run_simulation.starting_grid)
    print(f"{len(run_simulation.drivers_list)} Pilotes instanciés:")

    for d in run_simulation.drivers_list:
        print(f" - {d.name} (Id={d.driver_id}) | Team={d.team.name}")
    
    print("\nDataFrame laps_recap :")
    print(run_simulation.laps_recap)
    

    print("\n=== Fin du test ===")



IndentationError: expected an indented block after 'if' statement on line 77 (587191399.py, line 79)

NameError: name 'N0' is not defined

In [9]:
def main():
    db_path = "F1_timingdata_2014_2019.sqlite"
    data_loader = DataLoader(db_path=db_path)
    dataframes = data_loader.load_data()
    mercedes=Team("Mercedes")
    pit_stop = PitStop(
        team=mercedes, 
        gp_location="SaoPaulo",
        season=2016, 
        dataframes=dataframes
    )

    pit_stop.calculate_best_pit_stop_duration()
    print("Average Min Pit Stop Duration:", pit_stop.avg_min_pit_stop_duration)

    calculated_duration = pit_stop.calculate_pit_stop_duration()
    print("Calculated Pit Stop Duration:", calculated_duration)


if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"[ERROR] An exception occurred: {e}")


Average Min Pit Stop Duration: 22.886362499999997


,race_id,lapno,position,driver_id,laptime,racetime,gap,interval,compound,tireage,...,pitstopduration,nextcompound,startlapprog_vsc,endlapprog_vsc,age_vsc,startlapprog_sc,endlapprog_sc,age_sc,season,team
41870,39,12,5,3,110.969,1134.161,21.431,7.891,A4,14,...,21.531,A3,NaN,NaN,NaN,NaN,NaN,NaN,2016,Mercedes
41954,39,16,5,1,113.739,1523.223,21.749,1.405,A4,18,...,21.940,A2,NaN,NaN,NaN,NaN,NaN,NaN,2016,Mercedes
42905,40,13,1,3,102.553,1280.500,0.000,0.000,A4,15,...,24.934,A3,NaN,NaN,NaN,NaN,NaN,NaN,2016,Mercedes
42906,40,13,2,1,103.018,1299.084,18.584,18.584,A4,15,...,24.512,A2,NaN,NaN,NaN,NaN,NaN,NaN,2016,Mercedes
43177,40,28,3,1,101.221,2791.761,26.918,11.003,A2,15,...,25.030,A4,NaN,NaN,NaN,NaN,NaN,NaN,2016,Mercedes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61876,56,11,1,1,103.680,1143.220,0.000,0.000,A3,13,...,23.601,A3,NaN,NaN,NaN,NaN,NaN,NaN,2016,Mercedes
62280,56,31,1,1,121.052,3253.418,0.000,0.000,A3,20,...,23.575,A2,0.385,1.0,0.615,NaN,NaN,NaN,2016,Mercedes
62281,56,31,2,3,125.712,3268.698,15.280,15.280,A2,21,...,24.149,A2,0.286,1.0,0.714,NaN,NaN,NaN,2016,Mercedes
63101,57,17,1,1,86.713,1509.941,0.000,0.000,A3,19,...,21.709,A2,NaN,NaN,NaN,NaN,NaN,NaN,2016,Mercedes


Calculated Pit Stop Duration: 17.569961490287838


In [238]:
if __name__ == "__main__":
    # Step 1: Load the data using DataLoader
    data_loader = DataLoader(db_path="F1_timingdata_2014_2019.sqlite")
    dataframes = data_loader.load_data()

    # Step 2: Set up the Run instance
    season_test = 2019
    gp_location_test = "SaoPaulo"
    run_simulation = Run(
        season=season_test,
        gp_location=gp_location_test,
        dataframes=dataframes
    )

    # Step 3: Select a driver and set pit stop info
    test_driver = run_simulation.drivers_list[0]  # Take the first driver for testing
    test_driver.pit_stops_info = {
        0: {"compound": "Soft", "pit_stop_lap": 10},  # Example strategy
        1: {"compound": "Medium", "pit_stop_lap": 30}
    }
    test_driver.next_pit_stop = 0
    test_driver.tire_age = 25  # Force a pit stop condition

    # Step 4: Simulate a pit stop at lap 10
    lap = 10
    print(f"Before pit stop: {test_driver.name}")
    print(f"  Tire Age: {test_driver.tire_age}")
    print(f"  Cumulative Lap Time: {test_driver.cumulative_lap_time}")

    pit_duration = run_simulation.pit_stop(test_driver, lap)

    print(f"Pit stop duration: {pit_duration:.2f} seconds")
    print(f"After pit stop: {test_driver.name}")
    print(f"  Tire Age: {test_driver.tire_age}")
    print(f"  Compound: {test_driver.compound}")
    print(f"  Cumulative Lap Time: {test_driver.cumulative_lap_time}")
    print(f"  Next Pit Stop Index: {test_driver.next_pit_stop}")

    print("\n=== End of Test ===")


In [2]:
class DataLoader:
    """
    Class responsible for loading data from an SQLite database.
    """

    def __init__(self, db_path: str):
        self.db_path = db_path
        self.dataframes = {}

    def load_data(self) -> dict:
        """
        Load data from the SQLite database and store it in a dictionary.

        Returns:
            dict: A dictionary where keys are table names and values are DataFrames.
        """
        connection = sqlite3.connect(self.db_path)
        tables = [
            "drivers",
            "fcyphases",
            "laps",
            "qualifyings",
            "races",
            "retirements",
            "starterfields",
        ]
        self.dataframes = {
            table: pd.read_sql_query(f"SELECT * FROM {table}", connection)
            for table in tables
        }
        connection.close()
        return self.dataframes



In [3]:

class FuelAndTireModel:
    def get_parameters(self, driver_name: str, season: int):
        """
        Example method that returns fuel and tire parameters.
        """
        return {
            "fuel_efficiency": 0.98,
            "tire_degradation_rate": 0.05
        }


class DNFModel:
    def get_parameters(self, driver_name: str, season: int):
        """
        Example method that returns probabilities for DNF.
        """
        return 0.2, 0.2  # (prob_accident, prob_failure)



In [4]:

class Team:
    def __init__(self, name: str):
        self.name = name


class TeamRegistry:
    """
    Registry (or cache) to ensure a single instance of Team per unique name.
    """
    _teams_cache = {}

    @classmethod
    def get_team(cls, name: str) -> Team:
        if name not in cls._teams_cache:
            cls._teams_cache[name] = Team(name)
        return cls._teams_cache[name]



In [54]:

class Driver:
    def __init__(self, season: int, dataframes: dict, name: str):
        self.season = season
        self.dataframes = dataframes
        self.name = name

        self.team = None
        self.driver_id = None
        self.initials = None

        # Simulation attributes
        self.position = None
        self.cumulative_lap_time = 0
        self.compound = "medium"
        self.tire_age = 0
        self.next_pit_stop = 1

        # Strategy and DNF attributes
        self.pit_stops_info = {
            1: {
                "compound": "medium",
                "pitstop_interval": [10, 20],
                "pit_stop_lap": 15  # ou aléatoire entre les bornes de pitstop_interval
            },
            2: {
                "compound": "hard",
                "pitstop_interval": [25, 35],
                "pit_stop_lap": 30
            }
            # Et ainsi de suite pour les autres arrêts
        } # Modifer None

        self.accident_dnf_probability = None
        self.failure_dnf_probability = None
        self.accident_dnf_lap = None
        self.failure_dnf_lap = None
        self.earliest_dnf_lap = None
        self.alive = True

        # Fuel & Tire attributes
        self.fuel_tire_parameters = None
        
    def get_driver_parameters(self):
        drivers_df = self.dataframes["drivers"]
        starterfields_df = self.dataframes["starterfields"]
        races_df = self.dataframes["races"]

        # Get driver ID and initials
        driver_row = drivers_df[drivers_df["name"] == self.name]
        if driver_row.empty:
            raise ValueError(f"Driver '{self.name}' not found in 'drivers' table.")

        self.driver_id = driver_row.iloc[0]["id"]
        self.initials = driver_row.iloc[0]["initials"]

        # Get team
        merged_data = starterfields_df.merge(
            races_df,
            left_on="race_id",
            right_on="id",
            suffixes=("_sf", "_races")
        )
        team_row = merged_data[
            (merged_data["driver_id"] == self.driver_id) &
            (merged_data["season"] == self.season)
        ]
        if not team_row.empty:
            self.team = TeamRegistry.get_team(team_row.iloc[0]["team"])

        # Get DNF probabilities
        dnf_model = DNFModel()
        self.accident_dnf_probability, self.failure_dnf_probability = dnf_model.get_parameters(
            driver_name=self.name,
            season=self.season
        )

        # Get fuel and tire parameters
        fuel_tire_model = FuelAndTireModel()
        self.fuel_tire_parameters = fuel_tire_model.get_parameters(
            driver_name=self.name,
            season=self.season
        )

    def update_status(self, current_lap: int):
        if self.alive and self.earliest_dnf_lap == current_lap:
            self.alive = False

    def update_info(self):
        self.tire_age+=1
        #  #  



In [ ]:

class Run:
    def __init__(self, season: int, gp_location: str, dataframes: dict):
        # Reste : 
            # - Initialiser safety_car_laps
            # - Créer compute lap time
        self.season = season
        self.gp_location = gp_location
        self.dataframes = dataframes

        self.safety_car_laps=[12,27]

        self.number_of_laps = None
        self.drivers_list = []
        self.starting_grid = None

        self.laps_summary = pd.DataFrame(
            columns=["lap", "driver_id", "position", "lap_time", "cumulative_lap_time"]
        )

        self._initialize_parameters()
        

    def run(self):
        for lap in range(1,self.number_of_laps):  
            for driver in self.drivers_list:
                lap_time=0
                # State and change the driver status 
                driver.update_status(lap)
                driver.update_info()
                # Normal lap
                lap_time+=self.compute_lap_time(driver,lap)
                # Pit Stop 
                pit_stop_time=self.pit_stop(driver,lap)
                lap_time+=pit_stop_time
                driver.cumulative_lap_time+=lap_time  # Modifier  
            # # Overtaking
            for driver in self.drivers_list:
                self.get_driver_position(driver)
         # # Aggrémenter le recap des tours
            self.laps_summary = pd.concat([
                self.laps_summary,
                pd.DataFrame([{
                    "lap": lap,
                    "driver_id": driver.driver_id,
                    "position": driver.position,
                    "lap_time": lap_time,
                    "cumulative_lap_time": driver.cumulative_lap_time
                }])
            ], ignore_index=True)

    def get_driver_position(self, driver):
        """
        Calcule la position d'un pilote en fonction des temps cumulés.
        """
        sorted_drivers = sorted(
            [d for d in self.drivers_list if d.alive],
            key=lambda d: d.cumulative_lap_time
        )
        for idx, d in enumerate(sorted_drivers, start=1):
            if d == driver:
                driver.position=idx

    def compute_lap_time(self,driver,nolap):

        # Coder compute_lap_time
        # Tire Et fuel model
    #       lap_time=0
    #     if driver.alive:
    #         # FUel & TIre Model
    #         model=driver.model
    #         data_for_predictions=driver.select_features(nolap)
    #         lap_time+=model.predict(data_for_predictions) # Fuel level, tire age, coupound

    #     if nolap.isin(self.safetycar_laps):
    #         lap_time*=1.2

    #     return(lap_time)
        return(0.2)


    def _initialize_parameters(self):
        
        races_df = self.dataframes["races"]
        qualifyings_df = self.dataframes["qualifyings"]

        # Get number of laps
        race_row = races_df[
            (races_df["location"] == self.gp_location) & 
            (races_df["season"] == self.season)
        ]
        if race_row.empty:
            raise ValueError(f"No race found for '{self.gp_location}' in season {self.season}.")
        self.number_of_laps = race_row.iloc[0]["nolapsplanned"]

        # Build starting grid
        merged_data = qualifyings_df.merge(
            races_df, 
            left_on="race_id", 
            right_on="id", 
            suffixes=("_qualifying", "_race")
        )
        qualifying_rows = merged_data[
            (merged_data["season"] == self.season) & 
            (merged_data["location"] == self.gp_location)
        ]
        if qualifying_rows.empty:
            raise ValueError(f"No qualifying data found for '{self.gp_location}' in season {self.season}.")
        sorted_rows = qualifying_rows.sort_values(by="position")
        self.starting_grid = list(zip(sorted_rows["driver_id"], sorted_rows["position"]))

        # Instantiate drivers
        driver_ids = sorted_rows["driver_id"].unique().tolist()
        drivers_df = self.dataframes["drivers"]

        for driver_id in driver_ids:
            driver_row = drivers_df[drivers_df["id"] == driver_id]
            if driver_row.empty:
                continue

            driver_name = driver_row.iloc[0]["name"]
            driver_obj = Driver(self.season, self.dataframes, driver_name)
            driver_obj.get_driver_parameters()
            self.drivers_list.append(driver_obj)
    
    def pit_stop(self, driver, current_lap):
        """
        Simulates a pit stop for a driver if the conditions are met.

        Args:
            driver (Driver): The driver performing the pit stop.
            current_lap (int): The current lap of the race.
            safety_car_laps (list): List of laps where the safety car is active.

        Returns:
            float: Duration of the pit stop in seconds, or None if no pit stop is performed.
        """
        try:
            # Vérifier si un pit stop est prévu pour le pilote
            if driver.next_pit_stop in driver.pit_stops_info:
                pit_stop_data = driver.pit_stops_info[driver.next_pit_stop]

                # Vérifier si le tour actuel correspond aux conditions pour un arrêt
                is_pit_stop_lap = current_lap == pit_stop_data["pit_stop_lap"]
                is_safety_car_pit_stop = (
                    current_lap in self.safety_car_laps and current_lap in range(
                        pit_stop_data["pitstop_interval"][0], pit_stop_data["pitstop_interval"][1] + 1
                    )
                )

                if is_pit_stop_lap or is_safety_car_pit_stop:
                    # Effectuer le pit stop
                    pit_stop = PitStop(
                        team=driver.team,
                        gp_location=self.gp_location,
                        season=self.season,
                        dataframes=self.dataframes
                    )

                    # Calculer la durée moyenne du pit stop
                    pit_stop.calculate_best_pit_stop_duration()

                    # Simuler la durée du pit stop
                    calculated_duration = pit_stop.calculate_pit_stop_duration()

                    # Mettre à jour les informations du pilote
                    driver.tire_age = 0
                    driver.compound = pit_stop_data["compound"]
                    driver.next_pit_stop += 1

                    return calculated_duration

            # Si aucune condition de pit stop n'est remplie
            return 0

        except ValueError as e:
            print(f"Error during pit stop for driver {driver.name}: {e}")
            return None



    def simulate_dnf_lap(self, driver):
        driver.accident_dnf_lap = (
            np.random.randint(1, self.number_of_laps + 1)
            if np.random.binomial(1, driver.accident_dnf_probability)
            else None
        )
        driver.failure_dnf_lap = (
            np.random.randint(1, self.number_of_laps + 1)
            if np.random.binomial(1, driver.failure_dnf_probability)
            else None
        )
        potential_dnf_laps = [
            lap for lap in (driver.accident_dnf_lap, driver.failure_dnf_lap) if lap is not None
        ]
        driver.earliest_dnf_lap = min(potential_dnf_laps, default=None)


if __name__ == "__main__":
    data_loader = DataLoader(db_path="F1_timingdata_2014_2019.sqlite")
    dataframes = data_loader.load_data()

    run_simulation = Run(season=2019, gp_location="SaoPaulo", dataframes=dataframes)

    for driver in run_simulation.drivers_list:
        run_simulation.simulate_dnf_lap(driver)
        print(f"Driver: {driver.name}")
        print(f"  Accident lap: {driver.accident_dnf_lap}")
        print(f"  Failure lap: {driver.failure_dnf_lap}")
        print(f"  First DNF lap: {driver.earliest_dnf_lap}")


Driver: Max Verstappen
  Accident lap: None
  Failure lap: None
  First DNF lap: None
Driver: Sebastian Vettel
  Accident lap: None
  Failure lap: None
  First DNF lap: None
Driver: Lewis Hamilton
  Accident lap: None
  Failure lap: None
  First DNF lap: None
Driver: Charles Leclerc
  Accident lap: None
  Failure lap: None
  First DNF lap: None
Driver: Valtteri Bottas
  Accident lap: None
  Failure lap: None
  First DNF lap: None
Driver: Alexander Albon
  Accident lap: 55
  Failure lap: 67
  First DNF lap: 55
Driver: Pierre Gasly
  Accident lap: None
  Failure lap: None
  First DNF lap: None
Driver: Romain Grosjean
  Accident lap: 51
  Failure lap: None
  First DNF lap: 51
Driver: Kimi Raikkonen
  Accident lap: None
  Failure lap: None
  First DNF lap: None
Driver: Kevin Magnussen
  Accident lap: None
  Failure lap: None
  First DNF lap: None
Driver: Lando Norris
  Accident lap: None
  Failure lap: 5
  First DNF lap: 5
Driver: Daniel Ricciardo
  Accident lap: 35
  Failure lap: None
  F

In [65]:
if __name__ == "__main__":
    data_loader = DataLoader(db_path="F1_timingdata_2014_2019.sqlite")
    dataframes = data_loader.load_data()

    run_simulation = Run(season=2019, gp_location="SaoPaulo", dataframes=dataframes)
    run_simulation.run()

    # Affichez un résumé des résultats
    for driver in run_simulation.drivers_list:
        print(f"Driver: {driver.name}, Cumulative Lap Time: {driver.cumulative_lap_time}")
    display(run_simulation.laps_summary)

C:\Users\mateo\AppData\Local\Temp\ipykernel_20704\3219540205.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.laps_summary = pd.concat([


Driver: Max Verstappen, Cumulative Lap Time: 65.4582534659668
Driver: Sebastian Vettel, Cumulative Lap Time: 59.92451969632687
Driver: Lewis Hamilton, Cumulative Lap Time: 64.17821718410397
Driver: Charles Leclerc, Cumulative Lap Time: 63.109187681092976
Driver: Valtteri Bottas, Cumulative Lap Time: 63.12495737931576
Driver: Alexander Albon, Cumulative Lap Time: 60.39027802657641
Driver: Pierre Gasly, Cumulative Lap Time: 63.54244730560151
Driver: Romain Grosjean, Cumulative Lap Time: 46.587857209435676
Driver: Kimi Raikkonen, Cumulative Lap Time: 56.56107101568633
Driver: Kevin Magnussen, Cumulative Lap Time: 48.19685378227069
Driver: Lando Norris, Cumulative Lap Time: 85.78821120575391
Driver: Daniel Ricciardo, Cumulative Lap Time: 74.14513405265362
Driver: Antonio Giovinazzi, Cumulative Lap Time: 85.65314577251054
Driver: Nico Hulkenberg, Cumulative Lap Time: 69.75724761881385
Driver: Sergio Perez, Cumulative Lap Time: 59.08717011047291
Driver: Daniil Kvyat, Cumulative Lap Time: 63.

,lap,driver_id,position,lap_time,cumulative_lap_time
0,1,25,20,0.2,0.200000
1,2,25,20,0.2,0.400000
2,3,25,20,0.2,0.600000
3,4,25,20,0.2,0.800000
4,5,25,20,0.2,1.000000
...,...,...,...,...,...
65,66,25,6,0.2,57.298597
66,67,25,6,0.2,57.498597
67,68,25,6,0.2,57.698597
68,69,25,6,0.2,57.898597


In [ ]:
    run_simulation.laps_summary

In [37]:
if __name__ == "__main__":
    data_loader = DataLoader(db_path="F1_timingdata_2014_2019.sqlite")
    dataframes = data_loader.load_data()

    run_simulation = Run(season=2019, gp_location="SaoPaulo", dataframes=dataframes)

    for driver in run_simulation.drivers_list:
        # Simulate pit stop
        pit_duration = run_simulation.pit_stop(driver,12)
        print(f"  Pit stop duration: {pit_duration:.2f} seconds" if pit_duration else "  Pit stop failed.")


  Pit stop duration: 23.94 seconds
  Pit stop duration: 21.21 seconds
  Pit stop duration: 22.22 seconds
  Pit stop duration: 44.90 seconds
  Pit stop duration: 25.54 seconds
  Pit stop duration: 31.68 seconds
  Pit stop duration: 24.72 seconds
  Pit stop duration: 21.15 seconds
  Pit stop duration: 21.26 seconds
  Pit stop duration: 21.55 seconds
  Pit stop duration: 20.11 seconds
  Pit stop duration: 21.05 seconds
  Pit stop duration: 20.16 seconds
  Pit stop duration: 25.01 seconds
  Pit stop duration: 19.05 seconds
  Pit stop duration: 22.61 seconds
  Pit stop duration: 19.68 seconds
  Pit stop duration: 16.88 seconds
  Pit stop duration: 15.90 seconds
  Pit stop duration: 20.11 seconds


In [7]:

if __name__ == "__main__":
    
    data_loader = DataLoader(db_path="F1_timingdata_2014_2019.sqlite")
    dataframes = data_loader.load_data()

    # Sélectionner une saison et un nom de GP
    season_test = 2019
    gp_location_test = "SaoPaulo"

    run_simulation = Run(
        season=season_test,
        gp_location=gp_location_test,
        dataframes=dataframes
    )

    for driver in run_simulation.drivers_list:
        run_simulation.simulate_dnf_lap(driver)
        print(f"Driver: {driver.name}")
        print(f"  Accident lap: {driver.accident_dnf_lap}")
        print(f"  Failure lap: {driver.failure_dnf_lap}")
        print(f"  First DNF lap: {driver.earliest_dnf_lap}")

    print("\n=== Fin du test ===")

Driver: Max Verstappen
  Accident lap: None
  Failure lap: 31
  First DNF lap: 31
Driver: Sebastian Vettel
  Accident lap: None
  Failure lap: 46
  First DNF lap: 46
Driver: Lewis Hamilton
  Accident lap: None
  Failure lap: None
  First DNF lap: None
Driver: Charles Leclerc
  Accident lap: None
  Failure lap: None
  First DNF lap: None
Driver: Valtteri Bottas
  Accident lap: None
  Failure lap: 43
  First DNF lap: 43
Driver: Alexander Albon
  Accident lap: None
  Failure lap: None
  First DNF lap: None
Driver: Pierre Gasly
  Accident lap: None
  Failure lap: None
  First DNF lap: None
Driver: Romain Grosjean
  Accident lap: 1
  Failure lap: None
  First DNF lap: 1
Driver: Kimi Raikkonen
  Accident lap: 26
  Failure lap: 40
  First DNF lap: 26
Driver: Kevin Magnussen
  Accident lap: None
  Failure lap: None
  First DNF lap: None
Driver: Lando Norris
  Accident lap: None
  Failure lap: None
  First DNF lap: None
Driver: Daniel Ricciardo
  Accident lap: 28
  Failure lap: 36
  First DNF l